---------

# Enhanced BiGram model

----------

### Objective is correctness and clarity of concepts and not efficiency

#### Author : Anuj



------
------

IMPROVEMENTS:     
1) Better Tokenization      

LEFTOVERS:   

1) ~~Tensor board graphs for loss~~  

2) Tensor board for network vis      
   (a)   ~~Draw the Plot~~   
   (b)   Interpret and Map the network to Code     

3) Is embedding good or not ?    

4) Embedding visualization ?   
    (a) ~~Plot the vector in an interactive graph~~     
    (b) ~~Plot vectors in Tensorboard~~    
    (c) ~~After every k epochs, take some random words and show their nearest neighbours~~     
    (d) Show vectors being pulled appart in different direction    
    (e) Analogy tasks     
    (f) validation set loss   

5) model perplexity     

6) Input vectors and output vectors     

OPTIMIZATIONS     

1) Better Optimizer     
2) negative sampling    
3) manage learning rate     
4) Param initializations      
     

------
------


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import sys
import tensorflow as tf
import numpy as np
from collections import Counter
import random

In [3]:
sys.path.append("../../data/")


In [4]:
pwd

'/home/piyush/Piyush/study/anthill/NLP_Bootcamp/module2/BiGram'

# Load the data file 

In [5]:
path = "./../../data/harry_potter_3.txt"
text = open(path).read()

raw_data = text.replace('\r\n', ' ')



In [6]:
tokens = raw_data.split()


## frequency counts

In [7]:
# count the frequency of tokens, pick top 100
counts = Counter(tokens)
top_100 = counts.most_common(100)

In [8]:
top_100

[('the', 4578),
 ('to', 2510),
 ('and', 2392),
 ('of', 1990),
 ('a', 1965),
 ('was', 1590),
 ('his', 1401),
 ('said', 1263),
 ('he', 1257),
 ('Harry', 1256),
 ('in', 1112),
 ('had', 865),
 ('at', 761),
 ('you', 717),
 ('--', 710),
 ('I', 706),
 ('it', 684),
 ('that', 667),
 ('on', 621),
 ('with', 620),
 ('as', 590),
 ('for', 487),
 ('He', 464),
 ('out', 427),
 ('Hermione', 402),
 ('Ron', 396),
 ('were', 385),
 ('they', 384),
 ('be', 376),
 ('him', 372),
 ('from', 356),
 ('up', 356),
 ('into', 353),
 ('Professor', 342),
 ('have', 331),
 ('but', 322),
 ('her', 321),
 ('--"', 319),
 ('The', 310),
 ('all', 285),
 ('back', 282),
 ('been', 264),
 ('very', 252),
 ('Harry,', 246),
 ('what', 245),
 ('"I', 241),
 ('who', 240),
 ('their', 237),
 ('looked', 234),
 ('them', 231),
 ('this', 222),
 ('down', 219),
 ('like', 214),
 ('about', 214),
 ('one', 214),
 ('she', 211),
 ('Lupin', 210),
 ('could', 209),
 ('got', 205),
 ('not', 201),
 ('an', 199),
 ('is', 199),
 ('over', 194),
 ('around', 190),
 

## Build token to Id mapping

In [9]:
word_to_id = {}

for token in set(tokens):
    word_to_id[token] = len(word_to_id)


In [10]:
id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))


# Convert Data to IDs

In [11]:
data = []

data = [word_to_id[tok] for tok in tokens]

In [12]:
print (tokens[:20])

for id in data[:20]:
    print (id_to_word[id]) 

['Harry', 'Potter', 'and', 'the', 'Prisoner', 'of', 'Azkaban', 'by', 'J.K.', 'Rowling', 'CHAPTER', 'ONE', 'OWL', 'POST', 'Harry', 'Potter', 'was', 'a', 'highly', 'unusual']
Harry
Potter
and
the
Prisoner
of
Azkaban
by
J.K.
Rowling
CHAPTER
ONE
OWL
POST
Harry
Potter
was
a
highly
unusual


## Build validation set - randomly choose a batch from these 100

In [13]:
top_100_ids = []

for key, count in top_100:
    top_100_ids.append(word_to_id[key])
    
    
#print top_100_ids

In [14]:
validation_size = 32
validation_set = random.sample(top_100_ids, validation_size)

In [15]:
#top_100_ids

## Now lets slice the data into training set

In [16]:
class BatchData():
    
    def __init__(self, batch_size, list_of_token_ids):
        self.batch_size_ = batch_size
        self.data_ = list_of_token_ids
        self.data_len_ = len(self.data_)
        self.global_pointer = 0
        
    def getBatch(self):
         
        if self.global_pointer + self.batch_size_ > self.data_len_:
            print ("Abort")
            
        X = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        Y = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        
        for i in range(self.batch_size_-1):
            
            X[i] = self.data_[self.global_pointer]
            Y[i] = self.data_[self.global_pointer+1]
            
            self.global_pointer = (self.global_pointer + 1)%self.data_len_
            
            
        return X, Y
    
    def reset(self):
        
        self.global_pointer = 0
    

In [17]:
#batch = BatchData(batch_size=32, list_of_token_ids=data)

In [18]:
#X, Y = batch.getBatch()

In [19]:
#print (X)
#print (Y)

In [20]:
#data[:64]

# Network

In [21]:
batch_size=32
num_batches = len(data)/batch_size

embedding_dims = 128
vocab_size = len(set(tokens))

In [22]:
X = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='X_var')
Y = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='Y_var')
valid_X = tf.Variable(validation_set, dtype=tf.int32, name='X_valid')

In [23]:
y_oh = tf.one_hot(indices=X, depth=vocab_size, name='Converting_Y_to_Y_oh')

In [24]:
print (X.get_shape())
print (Y.get_shape())
#print (y_oh.get_shape())

(32,)
(32,)


In [25]:
embedding_layer_1 = tf.Variable(tf.truncated_normal(shape=(vocab_size, embedding_dims),mean=0.0, stddev=1.0, dtype=tf.float32), name="Embeddings_Matrix")    
embeded = tf.nn.embedding_lookup(embedding_layer_1, ids=X, name="Embedding_LookUp")

In [26]:
embeded.get_shape()

TensorShape([Dimension(32), Dimension(128)])

In [27]:
#softmax weights, bias
W = tf.Variable(tf.truncated_normal(shape=(embedding_dims, vocab_size),mean=0.0, stddev=1.0, dtype=tf.float32), name="Softmax_Weights_Matrix")
b = tf.Variable(tf.zeros(shape=(vocab_size,)), name="Softmax_Bias_Vector")


In [35]:
logits = tf.add(tf.matmul(embeded, W, name="WX"), b, name="WX_plus_b")

#logits = tf.add(tf.matmul(embed, softmax_weights, name="WX"), softmax_bias, name="WX_plus_b")

loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_oh, name="Compute_Loss")
#mean_loss = tf.reduce_mean(loss)
mean_loss = tf.reduce_mean(loss, name="Compute_mean_loss")

tf.summary.scalar("mean_loss", mean_loss)

<tf.Tensor 'mean_loss_1:0' shape=() dtype=string>

In [36]:
print (logits.get_shape())
print (y_oh.get_shape())

(32, 15519)
(32, 15519)


In [37]:
optimizer = tf.train.GradientDescentOptimizer(0.5, name="Optimizer").minimize(mean_loss)

In [38]:
summary_op = tf.summary.merge_all()

In [39]:
#compute L2 norm for cosine similarity
norm = tf.sqrt(tf.reduce_sum(tf.square(embedding_layer_1), axis=1, keepdims=True))
normalised_embeddings = embedding_layer_1 / norm

# get validation set embeddings
validation_data_embeddings = tf.nn.embedding_lookup(normalised_embeddings, ids=valid_X, name="validation_embeddings_lookup")

# similarity score of validation embeddings w.r.t normalised= dot product between validation_data_embeddings and mornalised embeddings
similarity = tf.matmul(validation_data_embeddings, tf.transpose(normalised_embeddings))  # C.A = C x transpose(A)

# Training

In [40]:
num_of_epochs = 5
LOG_DIR = './bigram_chk_pts'

# A SIMPLE saver() to save the model
saver = tf.train.Saver()

batch = BatchData(batch_size=32, list_of_token_ids=data)

with tf.Session() as sess:
    
    # writer to write graph to tensorboard
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

    tf.global_variables_initializer().run()
    print ("initialised")

    for epoch_id in range(num_of_epochs):

        av_batch_loss = 0

        for batch_id in range(int(num_batches)):

            X_, Y_ = batch.getBatch()

            feed_dict = {}
            feed_dict[X] = X_
            feed_dict[Y] = Y_

            batch_loss, _, summary = sess.run([mean_loss, optimizer,summary_op], feed_dict=feed_dict)
            
            #writer.add_summary(batch_loss, epoch) 
            writer.add_summary(summary, global_step=epoch_id)

            av_batch_loss += batch_loss
            
            if batch_id % 10000 == 0:
                print ("For epoch = %d, batch id = %d, batch loss = %f" %(epoch_id, batch_id, batch_loss))
                
                #print validation data
                sim = similarity.eval() # compute similarity
                
                #iterate over each validation example
                for i in range(validation_size):
                    word = id_to_word[validation_set[i]]
                    top_k = 8
                    # sort indexes and pick top k. we take 1:top_k+1 since 0th top pick will the same word itself
                    nearest = (sim[i,:].argsort()[1:top_k+1])
                    log = '\t Nearest to %s : ' %word
                    for k in range(top_k):
                        nearest_word = id_to_word[nearest[k]]
                        log = '%s %s,' %(log, nearest_word)
                    print (log)
                        
                    
                    

        print ("\nFor epoch = %d, Av loss = %f\n\n" %(epoch_id, av_batch_loss/num_batches))
        
        batch.reset()
        
    save_path = saver.save(sess, LOG_DIR)
    print("Model saved in file: %s" % save_path)
        
        
        
        

initialised
For epoch = 0, batch id = 0, batch loss = 39.320831
	 Nearest to into :  knotted, Averting, Footsteps, between, couple, impressed.", "COME, vermin,
	 Nearest to to :  beak, extracted, bored,, "C'min.", it:, around.", windows,, alive....",
	 Nearest to eyes :  pie,, wobbling, weekend., celebrated., bleeding., Unbidden,, steak-and-, yet!",
	 Nearest to around :  "Harry!, symptoms, Spells,, Peppermint, Weird, throat, "Hermione!", gasped,
	 Nearest to He :  insolently, pitch, lead., Dudley's, "Going, COMFORTED, "See?", clip,
	 Nearest to But :  bright, waged, jaw, slipstream., course..., swig, tutting, growl;,
	 Nearest to but :  arrange, Rosmerta,, glee, Howler, presume, advanced...., "Everything, stood,,
	 Nearest to toward :  twelve,, knees., dear,", speaking., ill, end, 'ead?", "Full,
	 Nearest to only :  twitched, same, too, agony..., Arts,, reply,, Mrs,, short.,
	 Nearest to you :  cornflakes., rain-filled, Secure, Snape.", clearly, "Who'd, wet,, Friday,
	 Nearest to do :

KeyboardInterrupt: 

# Plot the Embeddings 

## Tensorboard way

In [41]:
#create metadata file

#reference: https://www.tensorflow.org/get_started/embedding_viz#metadata
LOG_DIR = './bigram_chk_pts'

file_name = LOG_DIR + '/metadata.tsv'
fp = open(file_name, 'w')

for i, v in enumerate(id_to_word):
    print (id_to_word[i])
    
    string = str(i) + '_' + id_to_word[i] + '\n'
    fp.write(string)
    print (string)
    
    #if i > 5:
    #    break
        
fp.close()

as
0_as

sneak
1_sneak

DAD!"
2_DAD!"

pressed
3_pressed

exasperatedly.
4_exasperatedly.

prod
5_prod

distance.
6_distance.

LEAVES
7_LEAVES

Diagon
8_Diagon

testily,
9_testily,

see.
10_see.

fist
11_fist

dissolving....
12_dissolving....

legs,
13_legs,

Quietly
14_Quietly

noticing.
15_noticing.

ABOUT!
16_ABOUT!

"Lumos!
17_"Lumos!

closing
18_closing

full.
19_full.

million
20_million

'There
21_'There

lazily
22_lazily

o'clock,
23_o'clock,

needed
24_needed

lying.
25_lying.

homework.
26_homework.

long-suffering
27_long-suffering

ride
28_ride

bombshell
29_bombshell

stunned.
30_stunned.

drop
31_drop

huge
32_huge

Playing
33_Playing

slamming.
34_slamming.

stagecoaches
35_stagecoaches

winged
36_winged

He-Who-Must-Not-Be-Named
37_He-Who-Must-Not-Be-Named

effort
38_effort

we?"
39_we?"

defeat
40_defeat

"because
41_"because

Black
42_Black

tattered
43_tattered

bottom
44_bottom

trouble!
45_trouble!

lessons;
46_lessons;

exploding
47_exploding

facing
48_facing

di

1252_eventual

enchanted,
1253_enchanted,

struck
1254_struck

moldy-
1255_moldy-

allies...
1256_allies...

term,"
1257_term,"

taste
1258_taste

field
1259_field

worry,"
1260_worry,"

teeth
1261_teeth

trunks,
1262_trunks,

places.
1263_places.

consented
1264_consented

muttering,
1265_muttering,

"Wow,
1266_"Wow,

car
1267_car

drawl,
1268_drawl,

came.
1269_came.

disappearance
1270_disappearance

thinks
1271_thinks

humming
1272_humming

tangle
1273_tangle

"What,
1274_"What,

"Excellent,
1275_"Excellent,

knees,
1276_knees,

condition
1277_condition

together.....
1278_together.....

Underbred."
1279_Underbred."

wrong!"
1280_wrong!"

Ron.
1281_Ron.

Granger!
1282_Granger!

One
1283_One

minute."
1284_minute."

cafes,
1285_cafes,

briefcase.
1286_briefcase.

rosettes,
1287_rosettes,

bedpans
1288_bedpans

James....
1289_James....

hitting
1290_hitting

family."
1291_family."

complicated-looking
1292_complicated-looking

happy,
1293_happy,

thing,
1294_thing,

barrier.
1295_bar

"Leave
2252_"Leave

soup
2253_soup

tea.
2254_tea.

precise
2255_precise

can
2256_can

sweaty
2257_sweaty

TURNED
2258_TURNED

line.
2259_line.

everyone's
2260_everyone's

Longbottom,"
2261_Longbottom,"

"Okay!"
2262_"Okay!"

Wealsey.
2263_Wealsey.

names
2264_names

filing
2265_filing

Levels).
2266_Levels).

charging
2267_charging

phenomenal
2268_phenomenal

mongrels!"),
2269_mongrels!"),

"Welcome,"
2270_"Welcome,"

SCUM!"
2271_SCUM!"

nervous,
2272_nervous,

"See!
2273_"See!

They're
2274_They're

Ron?"
2275_Ron?"

Stan,
2276_Stan,

statue,
2277_statue,

listening?
2278_listening?

U._
2279_U._

threw
2280_threw

on?"
2281_on?"

uncontrollably,
2282_uncontrollably,

arm's
2283_arm's

Still
2284_Still

frowned;
2285_frowned;

mistake,
2286_mistake,

reflecting
2287_reflecting

offered.
2288_offered.

keepin'
2289_keepin'

swig
2290_swig

birthday
2291_birthday

Hooch?
2292_Hooch?

abroad,
2293_abroad,

saving
2294_saving

HAVEN'T!"
2295_HAVEN'T!"

goals,
2296_goals,

holidays,"
2

entered
3502_entered

pop
3503_pop

"COME
3504_"COME

elected
3505_elected

"Strip
3506_"Strip

Way,
3507_Way,

blankly
3508_blankly

broomstick?"
3509_broomstick?"

corner.
3510_corner.

nervously
3511_nervously

TALONS
3512_TALONS

choked.
3513_choked.

cake,
3514_cake,

sloppy
3515_sloppy

tightening
3516_tightening

wild-looking
3517_wild-looking

scuttling
3518_scuttling

benches
3519_benches

doorknob
3520_doorknob

capture
3521_capture

bad,
3522_bad,

silence
3523_silence

broom;
3524_broom;

INFORMATION
3525_INFORMATION

sorry?"
3526_sorry?"

ages;
3527_ages;

mean..."
3528_mean..."

suitcase.
3529_suitcase.

hired
3530_hired

squealed
3531_squealed

bet
3532_bet

evil-tempered
3533_evil-tempered

feeding
3534_feeding

shout
3535_shout

luck!"
3536_luck!"

Catastrophes
3537_Catastrophes

cold
3538_cold

injured,"
3539_injured,"

porridge,
3540_porridge,

ribs.
3541_ribs.

"He's
3542_"He's

unable
3543_unable

He.
3544_He.

Supplies.
3545_Supplies.

DARK,
3546_DARK,

practice
3


barking
4752_barking

"Indeed!"
4753_"Indeed!"

lot,
4754_lot,

shining
4755_shining

shirtsleeves
4756_shirtsleeves

whispered
4757_whispered

muttered,
4758_muttered,

furballs
4759_furballs

food,
4760_food,

splash.
4761_splash.

article
4762_article

shadow.
4763_shadow.

classes.
4764_classes.

Lavender's
4765_Lavender's

stranded,
4766_stranded,

forehead,
4767_forehead,

formation
4768_formation

countless
4769_countless

thirdly,"
4770_thirdly,"

silence.
4771_silence.

tug,
4772_tug,

School
4773_School

teams
4774_teams

head!
4775_head!

occupants
4776_occupants

that
4777_that

disgruntled
4778_disgruntled

up
4779_up

climate...
4780_climate...

successful,
4781_successful,

holding
4782_holding

over...
4783_over...

"Ooooo,
4784_"Ooooo,

them?
4785_them?

about.
4786_about.

laughing
4787_laughing

mysteries
4788_mysteries

energy.
4789_energy.

part,
4790_part,

knee.
4791_knee.

candle."
4792_candle."

real,
4793_real,

him,"
4794_him,"

made
4795_made

explain.
4796

...

cloak?"
8501_cloak?"

celebrated.
8502_celebrated.

seen;
8503_seen;

fifty-foot-high
8504_fifty-foot-high

podium,
8505_podium,

'ow
8506_'ow

Many
8507_Many

index
8508_index

donkey
8509_donkey

well,"
8510_well,"

weeks
8511_weeks

jet-black
8512_jet-black

teaching
8513_teaching

three-
8514_three-

climbed,
8515_climbed,

indeed....
8516_indeed....

indignation
8517_indignation

tells
8518_tells

MOONY,
8519_MOONY,

daggers
8520_daggers

house!"
8521_house!"

international
8522_international

MAP
8523_MAP

"Calm
8524_"Calm

Scabbers's
8525_Scabbers's

explaining
8526_explaining

complaints
8527_complaints

reasons...
8528_reasons...

Peter."
8529_Peter."

Monsters
8530_Monsters

Potter?
8531_Potter?

contentedly
8532_contentedly

branches.
8533_branches.

And,
8534_And,

September
8535_September

debris
8536_debris

propel
8537_propel

SCHOOL
8538_SCHOOL

Chasers,
8539_Chasers,

interesting
8540_interesting

cauldronful,"
8541_cauldronful,"

roots,
8542_roots,

Flint,
8543_Flint,


magnified
10251_magnified

collide
10252_collide

Sites
10253_Sites

MASTER....
10254_MASTER....

Crookshanks.
10255_Crookshanks.

table.
10256_table.

shining.
10257_shining.

weather.
10258_weather.

bars....
10259_bars....

tell
10260_tell

forty-ten
10261_forty-ten

Voldemort's
10262_Voldemort's

burnings,
10263_burnings,

handed
10264_handed

OU...
10265_OU...

tricks?"
10266_tricks?"

greatest
10267_greatest

ready?"
10268_ready?"

happy.
10269_happy.

shortened
10270_shortened

aunts!"
10271_aunts!"

fingers,
10272_fingers,

indignantly.
10273_indignantly.

'Nooo...
10274_'Nooo...

told?"
10275_told?"

tears.
10276_tears.

greeting,
10277_greeting,

mouse
10278_mouse

going,"
10279_going,"

"Quite
10280_"Quite

shop.
10281_shop.

lips,
10282_lips,

myself
10283_myself

Overcome
10284_Overcome

"Rosmerta,
10285_"Rosmerta,

teacher!"
10286_teacher!"

boxes.
10287_boxes.

arms
10288_arms

leaping
10289_leaping

words,
10290_words,

cautiously
10291_cautiously

"Jordan!"
10292_"Jor

11750_transformed

worsened
11751_worsened

Office
11752_Office

open
11753_open

noisy
11754_noisy

Pettigrew
11755_Pettigrew

point.
11756_point.

M.
11757_M.

ate,
11758_ate,

him....
11759_him....

again..
11760_again..

pallid
11761_pallid

coolly.
11762_coolly.

Harry...
11763_Harry...

fellytone
11764_fellytone

mgoing
11765_mgoing

sunny
11766_sunny

shoving
11767_shoving

providing
11768_providing

Turn
11769_Turn

Completely
11770_Completely

he?
11771_he?

"Listen,"
11772_"Listen,"

detention."
11773_detention."

proper-sized
11774_proper-sized

(drunk,
11775_(drunk,

"Great,
11776_"Great,

closer...
11777_closer...

shriek
11778_shriek

enormous.
11779_enormous.

continued
11780_continued

perfect,
11781_perfect,

"So,
11782_"So,

other;
11783_other;

SHE'S
11784_SHE'S

herself...
11785_herself...

realized,"
11786_realized,"

matters
11787_matters

newly
11788_newly

reach
11789_reach

confirmed.
11790_confirmed.

kitchen
11791_kitchen

traitor.
11792_traitor.

purple
1179

guilt.
13250_guilt.

"Most
13251_"Most

bulldog.
13252_bulldog.

spat,
13253_spat,

lolling
13254_lolling

SPLATTER.
13255_SPLATTER.

stay,
13256_stay,

prevent
13257_prevent

age,
13258_age,

vast
13259_vast

signed,
13260_signed,

role,
13261_role,

Order
13262_Order

Split
13263_Split

Snap?"
13264_Snap?"

sustain
13265_sustain

pulled.
13266_pulled.

caught;
13267_caught;

trance,
13268_trance,

settling
13269_settling

damage
13270_damage

apprehensively.
13271_apprehensively.

robe
13272_robe

corridors,
13273_corridors,

inform
13274_inform

out.
13275_out.

greenhouses
13276_greenhouses

Patronus,
13277_Patronus,

leaving;
13278_leaving;

replaced
13279_replaced

noticed
13280_noticed

Black."
13281_Black."

sweater
13282_sweater

seeing.
13283_seeing.

turn
13284_turn

rooftops,
13285_rooftops,

boxes
13286_boxes

under
13287_under

form."
13288_form."

hat.
13289_hat.

title
13290_title

crying.
13291_crying.

own,
13292_own,

accepted
13293_accepted

discovery.
13294_discove

half
14750_half

Didn't
14751_Didn't

been...
14752_been...

asleep
14753_asleep

Hall,
14754_Hall,

with
14755_with

Oliver,"
14756_Oliver,"

offered
14757_offered

kinder
14758_kinder

scar,
14759_scar,

wheel
14760_wheel

call
14761_call

YOU!"
14762_YOU!"

institution
14763_institution

Humongous
14764_Humongous

wetter
14765_wetter

poked
14766_poked

beetles
14767_beetles

nods.
14768_nods.

sat
14769_sat

forbidden
14770_forbidden

,anted
14771_,anted

homework,
14772_homework,

idea?"
14773_idea?"

hasn't
14774_hasn't

simple...
14775_simple...

saintly
14776_saintly

afterwards.
14777_afterwards.

Cleansweep
14778_Cleansweep

missing,"
14779_missing,"

hole,
14780_hole,

glimmers
14781_glimmers

thin.
14782_thin.

doubt
14783_doubt

rule."
14784_rule."

holidays."
14785_holidays."

blushing.
14786_blushing.

ago!"
14787_ago!"

Lights
14788_Lights

"Lupin
14789_"Lupin

Shocking
14790_Shocking

(Ordinary
14791_(Ordinary

unbiased
14792_unbiased

comparing
14793_comparing

Pumpki

In [42]:
from tensorflow.contrib.tensorboard.plugins import projector
import os

summary_writer = tf.summary.FileWriter(LOG_DIR)

#get config params
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_layer_1.name  # this is where you connect your embedding tensor

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

# run your tensorboard
# tensorboard --logdir="./bigram_chk_pts" --port 6006